# Llama 3.2 LoRA fine-tuning with Unsloth

Large Language Models (LLMs) used in Retrieval-Augmented Generation (RAG) pipelines often struggle when the retrieved context doesn’t contain the information needed to answer a user’s question.
Smaller models, in particular, may hallucinate — producing confident but incorrect answers instead of acknowledging missing evidence.

In this notebook, we’ll walk through fine-tuning Llama 3.2 using the Unsloth library to help the model:

- Detect when the retrieved context is irrelevant or incomplete, and
- Respond appropriately (e.g., “I couldn’t find that information in the provided materials.”)

Beyond simply saying “I don’t know,” we’ll also encourage the model to explain why it cannot provide a reliable answer.
This approach makes the model’s behavior more transparent, trustworthy, and user-friendly — reducing hallucinations while keeping responses informative and grounded.

# 0. The Problem

In [1]:
from unsloth import FastModel

model, tokenizer = FastModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=2048,
    load_in_4bit=True,
    load_in_8bit=False,
    full_finetuning=False,
)

/home/lpozzi/Git/data-science-lectures/lora_finetune/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA RTX 2000 Ada Generation Laptop GPU. Num GPUs = 1. Max memory: 7.653 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [5]:
system_prompt = """
Sei un assistente utile e accurato. Rispondi solo utilizzando le informazioni presenti nel contesto fornito.
Se il contesto non contiene dati sufficienti per rispondere, dillo chiaramente e spiega brevemente perché.
Non fare supposizioni, attieniti alle informazioni nel contesto.
"""

# context = """Gli Aepyornis sono un genere estinto di giganteschi uccelli vissuti in Madagascar, appartenenti alla famiglia degli uccelli elefante (Aepyornithidae), che comprende anche i Mullerornis, più piccoli, anch'essi estinti. 
# Si ritiene che fossero tra i più grandi uccelli mai esistiti assieme ai Moa. Potevano misurare fino a 3 m e più d'altezza, per un peso di oltre mezza tonnellata. 
# Le loro uova avevano una circonferenza di oltre un metro ed una lunghezza di più di 35 cm; il loro volume era circa 160 volte quello di un uovo di gallina. 
# Il DNA dell'Aepyornis è stato estratto con successo dai resti di gusci d'uova da un gruppo di ricercatori australiani. 
# Descrizione Si ritiene che l'espressione "uccello elefante" derivi dal Milione di Marco Polo, che parlando del Madagascar riferiva: Gli Aepyornis furono molto probabilmente osservati dagli Arabi (che intrattenevano rapporti con le dinastie reali malgasce), e potrebbero essere correlati al mito del Roc (o alle sue evoluzioni più recenti). 
# In malgascio, questi animali venivano chiamati contextvorompatra, uccelli degli "Ampatri", un toponimo che identificava l'attuale regione di Androy, nel sud dell'isola. In Madagascar, tuttavia, non sono mai stati ritrovati esemplari fossili o viventi di elefanti, ed è dubbio che lo stesso toponimo Madagascar, utilizzato sul finire del XV secolo da Martin Behaim per indicare l'isola, sia in realtà nel Milione una corruzione di Mogadiscio. 
# Inoltre, la descrizione della tecnica predatoria del grifone contrasta col fatto che l'Aepyornis è inadatto al volo. Specie Quattro sono le specie attualmente ascritte al genere: Aepyornis hildebrandti, Burckhardt, 1893
# """

# question = "Qual è la principale causa della scomparsa degli Aepyornis?"

from datasets import load_dataset

ds = load_dataset("lopozz/UA4RAG-it")



context = ds["test"][5]["context"]
question = ds["test"][5]["question"]


user_prompt = f"Contesto:\n{context}\n\n{question}"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,  # Must add for generation
)

[print(f"{k}: {v}\n") for k, v in ds["test"][5].items()]

question: Quale era la principale caratteristica della struttura sociale dell'aristocrazia romana antica?

context: Aristocrazia
Laristocrazia (dal greco άριστος, àristos, "migliore" e κράτος, kratos, "comando") è una forma di governo nella quale poche persone (che secondo l'etimologia greca del termine dovrebbero essere i "migliori") controllano interamente lo Stato; secondo il pensiero platonico-aristotelico è una delle tre forme di governo, assieme a monarchia e timocrazia, mentre l'oligarchia è la sua forma degenerata; è stata, assieme all'oligarchia, tra le forme di governo più diffuse in Europa negli ultimi secoli, generalmente sotto forma di monarchie costituzionali, nelle quali il potere del sovrano è controllato da un parlamento composto da soli nobili. Note Bibliografia Giuseppe Rensi, Forme di governo del passato e dell'avvenire, Roma, 1945 Arturo Beccari, Il pensiero politico classico, Milano, 1949 Lawrence Stone, The crisis of the aristocracy (1558-1641), Oxford, 1965 Ales

[None, None, None]

In [6]:
num_samples = 10

for i in range(num_samples):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    # Generate
    output_ids = model.generate(
        **inputs,
        max_new_tokens=125,
        temperature=1.0,
        top_p=0.95,
        top_k=64,
        streamer=None,  # disable live streaming for capturing
    )

    # Decode
    generated_text = tokenizer.decode(
        output_ids[0][inputs["input_ids"].shape[-1] :], skip_special_tokens=True
    )
    print(f"\nRun: {i + 1} - {generated_text}")
    print("----" * 20)


Run: 1 - La principale caratteristica della struttura sociale dell'aristocrazia romana antica era la divisione del potere tra il console (o console) e il senato (o senato).
--------------------------------------------------------------------------------

Run: 2 - La principale caratteristica della struttura sociale dell'aristocrazia romana antica era il possesso di "maggior potere" o "supremazia" nella società, poiché il termine greco "αριστocracia" (aristocracia) indica letteralmente "governo dei migliori".
--------------------------------------------------------------------------------

Run: 3 - La principale caratteristica della struttura sociale dell'aristocrazia romana antica era la proprietà di immobili fondi e altre risorse, il che le permetteva di esercitare controllo economico e di influenza politica sulla società.
--------------------------------------------------------------------------------


KeyboardInterrupt: 

## 1. Load and Prepare the dataset

| role     | purpose                                                                                     |
| -------- | ------------------------------------------------------------------------------------------- |
| question | The user’s query to be answered.                             |
| context  | Retrieved passages/snippets provided to the model (may be empty or irrelevant).   |
| answer   | The model’s grounded output: a clear “not found”. |


We now use the `Llama3.2` format for conversation style finetunes. Llama3.2 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>
Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>
I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. 

In [7]:
print(tokenizer.chat_template)

{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- if strftime_now is defined %}
        {%- set date_string = strftime_now("%d %b %Y") %}
    {%- else %}
        {%- set date_string = "26 Jul 2024" %}
    {%- endif %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if tools is not none %}
    {{- "Environment: ipython\n" }}
{%- endif %}
{{- "Cutting Knowledge Date: December 2023\n" }}
{{- 

In [8]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {
        "role": "user",
        "content": "Quali sono i criteri per l'identificazione di una sottoclasse all'interno della classe \"Aves\"?",
    },
    {
        "role": "assistant",
        "content": "Non è possibile rispondere alla domanda in base al testo fornito.",
    },
    {"role": "user", "content": "Ok grazie!"},
]

tokenized_chat = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=False
)
print(tokenized_chat)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 18 Nov 2025

You are a friendly chatbot who always responds in the style of a pirate<|eot_id|><|start_header_id|>user<|end_header_id|>

Quali sono i criteri per l'identificazione di una sottoclasse all'interno della classe "Aves"?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Non è possibile rispondere alla domanda in base al testo fornito.<|eot_id|><|start_header_id|>user<|end_header_id|>

Ok grazie!<|eot_id|>


In [9]:
system_prompt = """
Sei un assistente utile e accurato. Rispondi solo utilizzando le informazioni presenti nel contesto fornito.
Se il contesto non contiene dati sufficienti per rispondere, dillo chiaramente e spiega brevemente perché.
Non fare supposizioni, attieniti alle informazioni nel contesto.
"""

ds = ds.map(
    lambda ex: {
        "messages": [
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": f"Contesto:\n{ex['context'].replace('Eventi, invenzioni e scoperte ', '')}\n\n{ex['question']}",
            },
            {"role": "assistant", "content": ex["answer"]},
        ]
    }
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [10]:
ds["train"]["messages"][0]

[{'content': '\nSei un assistente utile e accurato. Rispondi solo utilizzando le informazioni presenti nel contesto fornito.\nSe il contesto non contiene dati sufficienti per rispondere, dillo chiaramente e spiega brevemente perché.\nNon fare supposizioni, attieniti alle informazioni nel contesto.\n',
  'role': 'system'},
 {'content': 'Contesto:\nAltri progetti 05\n\nQuale paese ha sviluppato il primo sistema di comunicazione satellitare?',
  'role': 'user'},
 {'content': "Non è possibile rispondere alla domanda, anche solo con il testo fornito. Il testo non fornisce alcuna informazione specifica su quale paese abbia sviluppato il primo sistema di comunicazione satellitare. La domanda presuppone la conoscenza di un evento storico e di un progetto, ma il contesto non offre alcun dettaglio sull'origine del primo sistema.\n",
  'role': 'assistant'}]

In [11]:
ds = ds.map(
    lambda ex: {
        "text": tokenizer.apply_chat_template(
            ex["messages"], tokenize=False, add_generation_prompt=False
        )
    }
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [12]:
# We now have to apply the chat template for Gemma3 onto the conversations, and save it to text.
print(ds["train"]["text"][2])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 18 Nov 2025

Sei un assistente utile e accurato. Rispondi solo utilizzando le informazioni presenti nel contesto fornito.
Se il contesto non contiene dati sufficienti per rispondere, dillo chiaramente e spiega brevemente perché.
Non fare supposizioni, attieniti alle informazioni nel contesto.<|eot_id|><|start_header_id|>user<|end_header_id|>

Contesto:
Bishōnen
indica un canone estetico di bellezza maschile tipicamente giapponese. Il prefisso si riferisce specificamente alla bellezza femminile, a una bella donna. Il bishōnen può essere inteso in più di un modo: ragazzo magro e non molto muscoloso, con un mento affusolato e un'apparenza effeminata o androgina. Incarna l'ideale del giovane amante omosessuale; alcuni appassionati occidentali utilizzano questo termine per riferirsi a qualsiasi bel personaggio maschile, nonostante quest'uso (letteralmente parlando) sia impreciso. V

## 2. Prepare the model

In [1]:
model

NameError: name 'model' is not defined

In [13]:
model = FastModel.get_peft_model(
    model,
    r=128,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=128,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

Unsloth: Making `model.base_model.model.model` require gradients


In [14]:
!nvidia-smi

Wed Nov  5 20:45:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


|   0  NVIDIA RTX 2000 Ada Gene...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   54C    P3             18W /   55W |    3177MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+------------------------+----------------------+
                                                                                         
+-----------------------------------------------------------------------------------------+
| Processes:                                                                              |
|  GPU   GI   CI        PID   Type   Process name                              GPU Memory |
|        ID   ID                                                               Usage      |
|=========================================================================================|
|    0   N/A  N/A      2794      G   /usr/lib/xorg/Xorg                           

In [ ]:
from trl import SFTTrainer, SFTConfig

# Now let's train our model. We do 100 steps to speed things up, but you
# can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds["train"],
    eval_dataset=None,  # Can set up evaluation!
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,  # Use GA to mimic batch size!
        warmup_steps=5,
        num_train_epochs=1,  # Set this for 1 full training run.
        # max_steps = 100,
        learning_rate=2e-5,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.001,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir=None,
        report_to="none",
    ),
)

In [16]:
# We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.
# This helps increase accuracy of finetunes!

from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Let's verify masking the instruction part is done! Let's print the 100th row again.

In [ ]:
print([
        tokenizer.pad_token_id if x == -100 else x
        for x in trainer.train_dataset[2]["labels"]
    ])

tokenizer.decode(
    [
        tokenizer.pad_token_id if x == -100 else x
        for x in trainer.train_dataset[2]["labels"]
    ]
).replace(tokenizer.pad_token, " ")

'                                                                                                                                                                         Non è possibile rispondere a questa domanda basandosi sul testo fornito. Il testo non contiene alcuna informazione relativa al numero di città governate da Alessandro Severo durante la sua spedizione in Oriente. La domanda presuppone che il testo contenga dati specifici sulla sua amministrazione, ma non lo fa.<|eot_id|>'

In [19]:
import torch

gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX 2000 Ada Generation Laptop GPU. Max memory = 7.653 GB.
3.051 GB of memory reserved.


In [20]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 80 | Num Epochs = 1 | Total steps = 10
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 194,510,848 of 3,407,260,672 (5.71% trained)


Step,Training Loss
1,1.031100
2,1.100800
3,1.070100
4,0.984200
5,0.943900
6,0.893700
7,0.961600
8,0.759500
9,0.662200
10,1.000100


Unsloth: Will smartly offload gradients to save VRAM!


In [21]:
# Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime'] / 60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

86.0898 seconds used for training.
1.43 minutes used for training.
Peak reserved memory = 7.271 GB.
Peak reserved memory for training = 4.22 GB.
Peak reserved memory % of max memory = 95.008 %.
Peak reserved memory for training % of max memory = 55.142 %.


In [ ]:
# Save model locally

# model_path = 'path/to/model'
# model.save_pretrained(model_path)  # Local saving
# tokenizer.save_pretrained(model_path)

## 5&nbsp;&nbsp;Inference
**YOU MAY NEED TO RESTART THE KERNEL HERE TO CLEAR THE GPU**

In [ ]:
# load the LoRA adapters we just saved

# from unsloth import FastLanguageModel
# 
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name="/home/lpozzi/Git/data-science-lectures/lora_finetune/llama-3.2-RAG",  # YOUR MODEL YOU USED FOR TRAINING
#     max_seq_length=2048,
#     load_in_4bit=True,
# )

from unsloth import FastModel

model, tokenizer = FastModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=2048,
    load_in_4bit=True,
    load_in_8bit=False,
    full_finetuning=False,
)

model.load_adapter("lopozz/Llama-3.2-3B-Instruct-UA4RAG-it-adapters")



/home/lpozzi/Git/data-science-lectures/lora_finetune/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA RTX 2000 Ada Generation Laptop GPU. Num GPUs = 1. Max memory: 7.653 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/778M [00:00<?, ?B/s]

In [2]:
from datasets import load_dataset

system_prompt = """
Sei un assistente utile e accurato. Rispondi solo utilizzando le informazioni presenti nel contesto fornito.
Se il contesto non contiene dati sufficienti per rispondere, dillo chiaramente e spiega brevemente perché.
Non fare supposizioni, attieniti alle informazioni nel contesto.
"""

ds = load_dataset("lopozz/UA4RAG-it")
context = ds["test"][5]["context"]
question = ds["test"][5]["question"]

print(question)
print(context)

user_prompt = f"Contesto:\n{context}\n\n{question}"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

num_samples = 10

for i in range(num_samples):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    # Generate
    output_ids = model.generate(
        **inputs,
        max_new_tokens=125,
        temperature=0.3,
        streamer=None,
    )

    # Decode
    generated_text = tokenizer.decode(
        output_ids[0][inputs["input_ids"].shape[-1] :], skip_special_tokens=True
    )
    print(f"\nRun: {i + 1} - {generated_text}")
    print("----" * 20)

Quale era la principale caratteristica della struttura sociale dell'aristocrazia romana antica?
Aristocrazia
Laristocrazia (dal greco άριστος, àristos, "migliore" e κράτος, kratos, "comando") è una forma di governo nella quale poche persone (che secondo l'etimologia greca del termine dovrebbero essere i "migliori") controllano interamente lo Stato; secondo il pensiero platonico-aristotelico è una delle tre forme di governo, assieme a monarchia e timocrazia, mentre l'oligarchia è la sua forma degenerata; è stata, assieme all'oligarchia, tra le forme di governo più diffuse in Europa negli ultimi secoli, generalmente sotto forma di monarchie costituzionali, nelle quali il potere del sovrano è controllato da un parlamento composto da soli nobili. Note Bibliografia Giuseppe Rensi, Forme di governo del passato e dell'avvenire, Roma, 1945 Arturo Beccari, Il pensiero politico classico, Milano, 1949 Lawrence Stone, The crisis of the aristocracy (1558-1641), Oxford, 1965 Alessandro Barbero, L'ar

KeyboardInterrupt: 

In [4]:
# Check for catastrophic forgetting!

from datasets import load_dataset

ds = load_dataset("ReDiX/wikipediaQA-ita")

system_prompt = """
Sei un assistente utile e accurato. Rispondi solo utilizzando le informazioni presenti nel contesto fornito.
Se il contesto non contiene dati sufficienti per rispondere, dillo chiaramente e spiega brevemente perché.
Non fare supposizioni, attieniti alle informazioni nel contesto.
"""


num_samples = 10

for i in range(num_samples):
    context = ds["train"][i]["context"]
    question = ds["train"][i]["question"]

    user_prompt = f"Contesto:\n{context}\n\n{question}"
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    # Generate
    output_ids = model.generate(
        **inputs,
        max_new_tokens=125,
        temperature=0.3,
        streamer=None,
    )

    # Decode
    generated_text = tokenizer.decode(
        output_ids[0][inputs["input_ids"].shape[-1] :], skip_special_tokens=True
    )
    print(question)
    print(f"\nRun: {i + 1} - {generated_text}")
    print("----" * 20)

Cosa è un organo a pompa?

Run: 1 - Un organo a pompa è un tipo di organo a serbatoio d'aria che comprende una o più tastiere, manuale e due pedali per azionare i mantici per l'aria.
--------------------------------------------------------------------------------
In quale famiglia di aerofoni ad ancia libera rientra l'organo a pompa?

Run: 2 - L'organo a pompa rientra nella famiglia di aerofoni ad ancia libera "serie di aerofoni ad ancia libera" (codice 412.132)
--------------------------------------------------------------------------------
Ci sono due tipi di organi a pompa: l'organo ad aria compressa e l'organo ad aria aspirata. Quali sono i nomi comuni di questi due tipi di organi?

Run: 3 - I nomi comuni per questi due tipi di organi a pompa sono: 

*   Armonium (o armonio)
*   Suction reed organ (o organo ad aria aspirata)
--------------------------------------------------------------------------------
Cosa significa antropologia?

Run: 4 - L'antropologia è una branca della scien

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
import os

token = os.environ["HUGGINGFACE_HUB_TOKEN"]
model_path = 'path/to/model'
# Merge to 16bit
if False:
    model.save_pretrained_merged(
        "model_path", tokenizer, save_method="merged_16bit"
    )
if False:  # Pushing to HF Hub
    model.push_to_hub_merged(
        "hf_user/model_path", tokenizer, save_method="merged_16bit", token=""
    )

# Merge to 4bit
if False:
    model.save_pretrained_merged(
        "model_path",
        tokenizer,
        save_method="merged_4bit",
    )
if False:  # Pushing to HF Hub
    model.push_to_hub_merged(
        "hf_user/model_path", tokenizer, save_method="merged_4bit", token=""
    )

# Just LoRA adapters
if False:
    model.save_pretrained("model_path")
    tokenizer.save_pretrained("model_path")
if False:  # Pushing to HF Hub
    model.push_to_hub("hf_user/model_path", token=token)
    tokenizer.push_to_hub(
        "hf_user/model_path", token=token
    )

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Saved model to https://huggingface.co/lopozz/Llama-3.2-3B-Instruct-UA4RAG-it-adapters


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            